### My Final Capstone FourSquare Project

#### Description of the problem and a discussion of the background:

> <b>Problem Statement:</b>
Finding a home suitable to personal tastes in relative to price expectation is getting harder and harder every other day. This research is done for real estate agencies to find houses for the people whom are eager to migrate or move into Toronto, CA from New York that are living in similar places.
 
> <b>Background:</b> Toronto is a city in Canada growing fast and is provincial capital of Ontario and the most populous city in Canada. A lot of people has the curiosity to discover what Toronto holds for them. Healtcare and wages are also well above expectations where for the most people in US which makes this city a target. Since immigration to Canada is more welcoming than most countries <a href='http://www.cic.gc.ca/english/helpcentre/results-by-topic.asp?top=23'>according to here</a>; Toronto has the most immigration ratio compared to other cities in Canada. Toronto is a culturally rich and diverse city as a result. Low crime rates and great cusine also a deciding factor for many and New Yorkers too. This paper is helping agencies to boost on these kind of factors, diversities, likes when there is a potention customer. My solution is creating an algorithm to find the best possible house for the customer according to where he lives and like in his/her homeplace (New York in our case) and makes a suggestion using machine learning and geo locator services. 

#### A description of the data and how it will be used to solve the problem.

> Toronto and New York location data will be used to compare the neighborhoods, and then we rank the neighborhoods from the selected boroughs for the cities.
>> I used the previous assignments to retrieve the neighborhood and geo co-ordinates for New York and Toronto. 
>> Unsupervised learning is used to cluster the neighborhoods of the two cities. 
>> Data sources are <a href='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'>Toronto Wikipedia WebPage </a> and New York.
>> Geo locations for Toronto will use following <a href='https://cocl.us/Geospatial_data'>source file</a>. 

>> Implementation logic is as follows:

>> <ul>
    <li>Create a dataset that holds the Geo-cordinates for Toronto's neighborhood</li>
    <li>Use Foursquare APIs to get venues for each of the Toronto's neighborhood</li>
    <li>Sort through the data to identify top 10 common venue categories for each of the Toronto's neighborhood</li>
    <li>Perform the above for New York's neighborhoods</li>
    <li>This step wil be the input. Select a neighborhood in NY for which we are looking for similar places in Toronto</li>
    <li>Use K-means on dataset that has all Toronto's neighborhoods plus this neighhborhood</li>
    <li>Then find the cluster which has the NY neighborhood in it and list all Toronto neighborhoods there</li>
</ul>


#### <u>Create a dataset that holds the Geo-cordinates for Toronto's neighborhood</u>

In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

In [2]:
CLIENT_ID = 'LCMXFXT1EUDTROOBSWO5UPW4PE2F2RLQMVO5TRLFAXCAZXM0' # your Foursquare ID
CLIENT_SECRET = 'F0ZV2YCJVW2GMUVB3FS14HCTKXCLCBMDPEIXL2OKYKLGCRPL' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30

In [3]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source, 'lxml')
PO_Table = soup.find('table',class_='wikitable sortable')
df = pd.read_html(str(PO_Table))
df_PO = df[0]
df_PO1 = df_PO[df_PO['Borough']!='Not assigned']
df_PO2 = df_PO1.groupby(['Postcode','Borough'])['Neighbourhood'].apply(lambda x: ','.join(x)).reset_index()
df_PO2['Neighbourhood'] = df_PO2.apply(
    lambda row: row['Borough'] if (row['Neighbourhood']== 'Not assigned') else row['Neighbourhood'],
    axis=1
)
geo_coord_master = pd.read_csv('https://cocl.us/Geospatial_data')
df_PO3 = pd.merge(df_PO2, geo_coord_master, 
                   left_on='Postcode', right_on = 'Postal Code', how='inner')
df_PO4 = df_PO3[['Postcode','Borough','Neighbourhood','Latitude','Longitude']]
neighborhoods = df_PO4[df_PO4['Borough'].str.contains('Toronto', na=False)][['Borough','Neighbourhood','Latitude','Longitude']]
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 4 boroughs and 38 neighborhoods.


#### <u>Further explore Toronto's neighborhood</u>

In [11]:
!conda install -c conda-forge geopy
!conda install -c conda-forge folium 

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          90 KB

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0   conda-forge
    geopy:         1.20.0-py_0 conda-forge


geopy-1.20.0         | 57 KB     | ##################################### | 100% 
geographiclib-1.49   | 32 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done
Solving environment: done

# All requested packages

In [12]:
import folium # map rendering library
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe


In [13]:
address = 'Toronto, ON'
geolocator = Nominatim(user_agent="Toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
Toronto_data = neighborhoods.reset_index(drop=True)

# create map of Toronto using latitude and longitude values
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(Toronto_data['Latitude'], Toronto_data['Longitude'], Toronto_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto

#### <u>Use Foursquare APIs to get venues for each of the Toronto's neighborhood</u>

In [14]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [16]:
Toronto_venues = getNearbyVenues(names=Toronto_data['Neighbourhood'],
                                   latitudes=Toronto_data['Latitude'],
                                   longitudes=Toronto_data['Longitude']
                                  )

The Beaches
The Danforth West,Riverdale
The Beaches West,India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park,Summerhill East
Deer Park,Forest Hill SE,Rathnelly,South Hill,Summerhill West
Rosedale
Cabbagetown,St. James Town
Church and Wellesley
Harbourfront,Regent Park
Ryerson,Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide,King,Richmond
Harbourfront East,Toronto Islands,Union Station
Design Exchange,Toronto Dominion Centre
Commerce Court,Victoria Hotel
Roselawn
Forest Hill North,Forest Hill West
The Annex,North Midtown,Yorkville
Harbord,University of Toronto
Chinatown,Grange Park,Kensington Market
CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place,Underground city
Christie
Dovercourt Village,Dufferin
Little Portugal,Trinity
Brockton,Exhibition Place,Parkdale Village
High Park,The Junction South
Parkdale,Roncesvall

In [17]:
neighborhood_latitude = Toronto_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = Toronto_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = Toronto_data.loc[0, 'Neighbourhood'] # neighborhood name

# one hot encoding
Toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Toronto_onehot['Neighbourhood'] = Toronto_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [Toronto_onehot.columns[-1]] + list(Toronto_onehot.columns[:-1])
Toronto_onehot = Toronto_onehot[fixed_columns]

Toronto_onehot.head()



,Neighbourhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Yoga Studio
0,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"The Danforth West,Riverdale",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [19]:
Toronto_grouped = Toronto_onehot.groupby('Neighbourhood').mean().reset_index()

#### <u>Sort through the data to identify top 10 common venue categories for each of the Toronto's neighborhood</u>

In [20]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
Toronto_neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
Toronto_neighborhoods_venues_sorted['Neighbourhood'] = Toronto_grouped['Neighbourhood']

for ind in np.arange(Toronto_grouped.shape[0]):
    Toronto_neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)

Toronto_neighborhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Steakhouse,Pizza Place,Café,Coffee Shop,Hotel,Asian Restaurant,Neighborhood,Seafood Restaurant,Lounge,Smoke Shop
1,Berczy Park,Cocktail Bar,Coffee Shop,Beer Bar,Farmers Market,Seafood Restaurant,Café,Basketball Stadium,Bistro,Breakfast Spot,Clothing Store
2,"Brockton,Exhibition Place,Parkdale Village",Coffee Shop,Café,Breakfast Spot,Grocery Store,Italian Restaurant,Performing Arts Venue,Pet Store,Climbing Gym,Caribbean Restaurant,Restaurant
3,Business Reply Mail Processing Centre 969 Eastern,Yoga Studio,Spa,Gym / Fitness Center,Garden Center,Garden,Fast Food Restaurant,Farmers Market,Light Rail Station,Comic Shop,Park
4,"CN Tower,Bathurst Quay,Island airport,Harbourf...",Airport Service,Airport Lounge,Airport Terminal,Harbor / Marina,Sculpture Garden,Airport Food Court,Airport Gate,Bar,Boat or Ferry,Boutique


#### <u>Perform the above for New York's neighborhoods</u>
<ul>
    <li>Create a dataset that holds the Geo-cordinates for NY's neighborhood</li>
    <li>Use Foursquare APIs to get venues for each of the NY's neighborhood</li>
    <li>Sort through the data to identify top 10 common venue categories for each of the NY's neighborhood</li>
</ul>

In [ ]:
!conda install -c conda-forge urllib3

In [51]:
import urllib3
urllib3.disable_warnings()

In [81]:

import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share your notebook.
client_38a8b1c345b24ebfb427cc0c04e00208 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='lesD155ERwXL9Nq-gfbbNxUMS5wdoKklYIgsPZ6z9KJq',
    ibm_auth_endpoint="https://iam.eu-de.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.eu-geo.objectstorage.service.networklayer.com')

body = client_38a8b1c345b24ebfb427cc0c04e00208.get_object(Bucket='capstonefinal-donotdelete-pr-s9hwh7rhyith7i',Key='newyork_data.json')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object 

if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

# Since JSON data can be semi-structured and contain additional metadata, it is possible that you might face an error during data loading.
# Please read the documentation of 'pandas.read_json()' and 'pandas.io.json.json_normalize' to learn more about the possibilities to adjust the data loading.
# pandas documentation: http://pandas.pydata.org/pandas-docs/stable/io.html#io-json-reader
# and http://pandas.pydata.org/pandas-docs/stable/generated/pandas.io.json.json_normalize.html

# df_data_1 = pd.read_json(body, orient='values')
# df_data_1.head()



In [79]:
http = urllib3.PoolManager()

url = 'https://github.com/ibmcloud1/Coursera_Capstone/blob/master/newyork_data.json'
response = http.request('GET', url)
soup = BeautifulSoup(response.data)
soup.head()

[<meta charset="utf-8"/>,
 <link href="https://github.githubassets.com" rel="dns-prefetch"/>,
 <link href="https://avatars0.githubusercontent.com" rel="dns-prefetch"/>,
 <link href="https://avatars1.githubusercontent.com" rel="dns-prefetch"/>,
 <link href="https://avatars2.githubusercontent.com" rel="dns-prefetch"/>,
 <link href="https://avatars3.githubusercontent.com" rel="dns-prefetch"/>,
 <link href="https://github-cloud.s3.amazonaws.com" rel="dns-prefetch"/>,
 <link href="https://user-images.githubusercontent.com/" rel="dns-prefetch"/>,
 <link crossorigin="anonymous" href="https://github.githubassets.com/assets/frameworks-da88be2d386821a8fa8de15789a23d23.css" integrity="sha512-kBwTI/PLZvQ8eHdCgA2vpn0uJe4kEM/Ft7Zz5SAkc9BAifCxBxQS4VTaEIDJpVrbrlAhCeRJvf7DQWaA4dEUUQ==" media="all" rel="stylesheet"/>,
 <link crossorigin="anonymous" href="https://github.githubassets.com/assets/site-f26347f7a4c8de793412afd3a9311ad1.css" integrity="sha512-oKHAxBIZ1VHILxQ+n0/i7KnjtyFVW1g/THCECjxCzGvDZ1ejVta

In [ ]:
import requests
import json
results = requests.get(url).json()
results

In [112]:
import json # library to handle JSON files
with open(response.data) as json_data:
    newyork_data = json.load(json_data)

# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)
neighborhoods_data = newyork_data['features']

for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

OSError: [Errno 36] File name too long: b'\n\n\n\n\n\n<!DOCTYPE html>\n<html lang="en">\n  <head>\n    <meta charset="utf-8">\n  <link rel="dns-prefetch" href="https://github.githubassets.com">\n  <link rel="dns-prefetch" href="https://avatars0.githubusercontent.com">\n  <link rel="dns-prefetch" href="https://avatars1.githubusercontent.com">\n  <link rel="dns-prefetch" href="https://avatars2.githubusercontent.com">\n  <link rel="dns-prefetch" href="https://avatars3.githubusercontent.com">\n  <link rel="dns-prefetch" href="https://github-cloud.s3.amazonaws.com">\n  <link rel="dns-prefetch" href="https://user-images.githubusercontent.com/">\n\n\n\n  <link crossorigin="anonymous" media="all" integrity="sha512-kBwTI/PLZvQ8eHdCgA2vpn0uJe4kEM/Ft7Zz5SAkc9BAifCxBxQS4VTaEIDJpVrbrlAhCeRJvf7DQWaA4dEUUQ==" rel="stylesheet" href="https://github.githubassets.com/assets/frameworks-da88be2d386821a8fa8de15789a23d23.css" />\n  <link crossorigin="anonymous" media="all" integrity="sha512-oKHAxBIZ1VHILxQ+n

In [93]:
address = 'New York, NY'
geolocator = Nominatim(user_agent="NY_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
NY_data = neighborhoods.reset_index(drop=True)
NY_data.head()

,Borough,Neighbourhood,Latitude,Longitude
0,East Toronto,The Beaches,43.676357,-79.293031
1,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188
2,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572
3,East Toronto,Studio District,43.659526,-79.340923
4,Central Toronto,Lawrence Park,43.728020,-79.388790


In [111]:
NY_venues = getNearbyVenues(names=NY_data['Neighbourhood'],
                                   latitudes=NY_data['Latitude'],
                                   longitudes=NY_data['Longitude']
                                  )

The Beaches
The Danforth West,Riverdale
The Beaches West,India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park,Summerhill East
Deer Park,Forest Hill SE,Rathnelly,South Hill,Summerhill West
Rosedale
Cabbagetown,St. James Town
Church and Wellesley
Harbourfront,Regent Park
Ryerson,Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide,King,Richmond
Harbourfront East,Toronto Islands,Union Station
Design Exchange,Toronto Dominion Centre
Commerce Court,Victoria Hotel
Roselawn
Forest Hill North,Forest Hill West
The Annex,North Midtown,Yorkville
Harbord,University of Toronto
Chinatown,Grange Park,Kensington Market
CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place,Underground city
Christie
Dovercourt Village,Dufferin
Little Portugal,Trinity
Brockton,Exhibition Place,Parkdale Village
High Park,The Junction South
Parkdale,Roncesvall

In [97]:
# one hot encoding
NY_onehot = pd.get_dummies(NY_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
NY_onehot['Neighbourhood'] = NY_venues['Neighbourhood'] 

# move neighborhood column to the first column
cols = list(NY_onehot)
# move the column to head of list using index, pop and insert
cols.insert(0, cols.pop(cols.index('Neighbourhood')))
# use ix to reorder
NY_onehot = NY_onehot.loc[:, cols]

NY_onehot.head()

,Neighbourhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Yoga Studio
0,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"The Danforth West,Riverdale",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [107]:
NY_grouped = NY_onehot.groupby('Neighbourhood').mean().reset_index()
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
NY_neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
NY_neighborhoods_venues_sorted['Neighbourhood'] = NY_grouped['Neighbourhood']

for ind in np.arange(NY_grouped.shape[0]):
    NY_neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(NY_grouped.iloc[ind, :], num_top_venues)

NY_neighborhoods_venues_sorted.head()



,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Steakhouse,Pizza Place,Café,Coffee Shop,Hotel,Asian Restaurant,Neighborhood,Seafood Restaurant,Lounge,Smoke Shop
1,Berczy Park,Cocktail Bar,Coffee Shop,Beer Bar,Farmers Market,Seafood Restaurant,Café,Basketball Stadium,Bistro,Breakfast Spot,Clothing Store
2,"Brockton,Exhibition Place,Parkdale Village",Coffee Shop,Café,Breakfast Spot,Grocery Store,Italian Restaurant,Performing Arts Venue,Pet Store,Climbing Gym,Caribbean Restaurant,Restaurant
3,Business Reply Mail Processing Centre 969 Eastern,Yoga Studio,Spa,Gym / Fitness Center,Garden Center,Garden,Fast Food Restaurant,Farmers Market,Light Rail Station,Comic Shop,Park
4,"CN Tower,Bathurst Quay,Island airport,Harbourf...",Airport Service,Airport Lounge,Airport Terminal,Harbor / Marina,Sculpture Garden,Airport Food Court,Airport Gate,Bar,Boat or Ferry,Boutique


## This step will be the input. Select a neighborhood in NY for which we are looking for similar in Toronto
> Here, the selection is <b>Chelsea</b> in New York as the input. So, I will be looking for places in Toronto that are similar to Chelsea, NY

In [108]:
NY_neighborhood_input = 'Chelsea'

In [109]:
NY_neighborhoods_venues_sorted[NY_neighborhoods_venues_sorted['Neighbourhood'] == 'Chelsea']

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue


In [110]:
NY_grouped[NY_grouped['Neighbourhood'] == 'Chelsea']

,Neighbourhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Yoga Studio


#### <u>Use K-means on dataset that has all Toronto's neighborhoods plus this neighhborhood</u>

In [20]:
combined_grouped = pd.concat([Toronto_grouped,NY_grouped[NY_grouped['Neighbourhood'] == 'Chelsea']], axis=0,sort=False, ignore_index=True)
combined_grouped.fillna(0, inplace=True)

In [21]:
combined_grouped[combined_grouped['Neighbourhood'] == 'Chelsea'] ## see if Chelsea is added to the DF

,Neighbourhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Veterinarian,Video Store,Warehouse Store,Waste Facility,Waterfront,Weight Loss Center,Whisky Bar,Wine Shop,Wings Joint,Women's Store
38,Chelsea,0.0,0.0,0.0,0.0,0.0,0.0,0.029412,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [22]:
combined_grouped.head()

,Neighbourhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Veterinarian,Video Store,Warehouse Store,Waste Facility,Waterfront,Weight Loss Center,Whisky Bar,Wine Shop,Wings Joint,Women's Store
0,"Adelaide,King,Richmond",0.0000,0.0000,0.0000,0.000,0.000,0.000,0.033333,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Berczy Park,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,"Brockton,Exhibition Place,Parkdale Village",0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Business Reply Mail Processing Centre 969 Eastern,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,"CN Tower,Bathurst Quay,Island airport,Harbourf...",0.0625,0.0625,0.0625,0.125,0.125,0.125,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### Run k-means

In [23]:
from sklearn.cluster import KMeans
# set number of clusters
kclusters = 10

combined_grouped_clustering = combined_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(combined_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 7, 1, 3, 7, 7, 1, 7, 3], dtype=int32)

In [24]:
# add clustering labels
combined_neighborhoods_venues_sorted = Toronto_neighborhoods_venues_sorted.append(NY_neighborhoods_venues_sorted[NY_neighborhoods_venues_sorted['Neighbourhood'] == 'Chelsea'])
combined_neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Toronto_merged = Toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Toronto_merged = Toronto_merged.join(combined_neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

Toronto_merged.head() # check the last columns!

,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Toronto,The Beaches,43.676357,-79.293031,9,Neighborhood,Other Great Outdoors,Health Food Store,Trail,Pub,Cuban Restaurant,Ethiopian Restaurant,Eastern European Restaurant,Dumpling Restaurant,Dog Run
1,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188,8,Greek Restaurant,Ice Cream Shop,Italian Restaurant,Yoga Studio,Bookstore,Restaurant,Spa,Diner,Juice Bar,Dessert Shop
2,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572,3,Park,Pet Store,Ice Cream Shop,Liquor Store,Sandwich Place,Burger Joint,Fast Food Restaurant,Fish & Chips Shop,Burrito Place,Steakhouse
3,East Toronto,Studio District,43.659526,-79.340923,7,Café,Coffee Shop,Bakery,Italian Restaurant,American Restaurant,Middle Eastern Restaurant,Stationery Store,Fish Market,Latin American Restaurant,Seafood Restaurant
4,Central Toronto,Lawrence Park,43.728020,-79.388790,4,Bus Line,Park,Swim School,Dance Studio,Falafel Restaurant,Ethiopian Restaurant,Eastern European Restaurant,Dumpling Restaurant,Dog Run,Discount Store


In [33]:
cluster_map = pd.DataFrame()
cluster_map['data_index'] = combined_grouped_clustering.index.values
cluster_map['cluster'] = kmeans.labels_

In [34]:
## get the index of Chelsea
combined_grouped[combined_grouped['Neighbourhood']=='Chelsea']

,Neighbourhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Veterinarian,Video Store,Warehouse Store,Waste Facility,Waterfront,Weight Loss Center,Whisky Bar,Wine Shop,Wings Joint,Women's Store
38,Chelsea,0.0,0.0,0.0,0.0,0.0,0.0,0.029412,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [32]:
cluster_map[cluster_map['data_index']==38] ##get the cluster that Chelsea is in

,data_index,cluster
38,38,1


In [27]:
cluster_map.groupby(['cluster']).count() ## double check if records are clustered

,data_index
cluster,
0,1
1,11
2,1
3,11
4,1
5,1
6,1
7,10
8,1


In [36]:
combined_grouped_with_cluster=combined_grouped.merge(cluster_map.set_index('data_index'), left_index=True, right_on='data_index')
combined_grouped_with_cluster[combined_grouped_with_cluster['cluster']==1]['Neighbourhood']

data_index
0                                Adelaide,King,Richmond
1                                           Berczy Park
3     Business Reply Mail Processing Centre 969 Eastern
7               Chinatown,Grange Park,Kensington Market
11                                           Davisville
12                                     Davisville North
15                          Dovercourt Village,Dufferin
30                              Ryerson,Garden District
32                      Stn A PO Boxes 25 The Esplanade
34                    The Annex,North Midtown,Yorkville
38                                              Chelsea
Name: Neighbourhood, dtype: object

In [ ]:
import matplotlib.cm as cm
import matplotlib.colors as colors
# create map
address = 'Toronto, ON'
geolocator = Nominatim(user_agent="Toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_merged['Latitude'], Toronto_merged['Longitude'], Toronto_merged['Neighbourhood'], Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### End of assignment